# **Keras Cats vs Dogs - Feature Extraction**

---

In this lesson, we learn how to use a pretrained network as a feature extractor. We'll then use those feautres as the input for our Logistic Regression Clasifier.
1. Download and Explore our data
2. Load our pretrained VGG16 Model
3. Extract our Features using VGG16
4. Train a LR Classifier using those features
5. Test some inferences

## **1. Download and Explore our data**

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.applications import VGG16, imagenet_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import tqdm
import os

In [ ]:
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/dogs-vs-cats.zip
!unzip -q dogs-vs-cats.zip
!unzip -q train.zip
!unzip -q test1.zip

--2023-11-04 13:19:31--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/dogs-vs-cats.zip
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 52.95.142.94, 3.5.245.154, 52.95.148.50, ...
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|52.95.142.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 851576689 (812M) [application/zip]
Saving to: ‘dogs-vs-cats.zip’

dogs-vs-cats.zip    100%[===================>] 812.13M  33.7MB/s    in 45s     

2023-11-04 13:20:17 (18.1 MB/s) - ‘dogs-vs-cats.zip’ saved [851576689/851576689]



### **Loading our data and it's labels into a dataframe**

There are many ways we can do this, this way is relatively simple to follow.

In [ ]:
filenames = os.listdir("./train")

categories = []

for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'class': categories
})
df.head()

,filename,class
0,dog.708.jpg,1
1,dog.10712.jpg,1
2,cat.3973.jpg,0
3,cat.3285.jpg,0
4,cat.1394.jpg,0


In [ ]:
len(df)

25000

## **2. Load our pretrained VGG16 Model**

In [ ]:
model = VGG16(weights="imagenet", include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

## **What exactly are we doing?**

We're taking the output of the last CONV-POOL layer (see below).

The output shape at this layer is **7 x 7 x 512**

![feat_extraction](https://appliedmachinelearning.files.wordpress.com/2021/05/ef54e-vgg16.png?w=612&zoom=2)
Image referenced from [here](https://appliedmachinelearning.blog/2019/07/29/transfer-learning-using-feature-extraction-from-trained-models-food-images-classification/)

## **3. Extract our Features using VGG16**

In [ ]:
image_names = os.listdir("./train")
image_paths = ["./train/"+ x for x in image_names][:8000]
labels = list(df['class'])

In [ ]:
batch_size = 32
image_features = []
image_labels = []

# loop over each batch
for i in range(0, len(image_paths)//batch_size):
  # extract our batches
  batch_paths = image_paths[i:i + batch_size]
  batch_labels = labels[i:i + batch_size]
  batch_images = []

  # iterate over each image and extract our image features
  for image_path in batch_paths:
    # load images using Keras's load_img() and resize to 224 x 244
    image = load_img(image_path, target_size = (224, 224))
    image = img_to_array(image)

    # We expand the dimensions and then subtract the mean RGB pixel intensity of ImageNet
    # using the imagenet_utils.preprocess_input() function
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)

    # append our image features to our batch list
    batch_images.append(image)

  # we take our batch of images and get in the right format with vstack
  batch_images = np.vstack(batch_images)

  # we then that batch and run it throuhg our predict function
  features = model.predict(batch_images, batch_size = batch_size)

  # we then take the output shape 7x7x512 and flatten it
  features = np.reshape(features,(-1, 7*7*512))

  # store our features and corresponding labels
  image_features.append(features)
  image_labels.append(batch_labels)

1/1 [==============================] - 0s 40ms/step


In [ ]:
# lets look at the image imageFeatures
print(image_features[0].shape)
image_features[0]

(32, 25088)


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.99640226,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        10.761819  ,  0.        ]], dtype=float32)

In [ ]:
image_labels[0]

[1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1]

## **4. Train a LR Classifier using those features**

First let's store our extracted feature info in a format that can loaded directly by sklearn.

In [ ]:
# take our list of batches and reduce the dimernsion so that it's now a list 25088 features x 8000 rows (8000 x 1 for our labels)
imageLabels_data =  [lb for label_batch in image_labels for lb in label_batch]
imageFeatures_data = [feature for feature_batch in image_features for feature in feature_batch]

# Convert to numpy arrays
image_labels_data = np.array(imageLabels_data)
image_features_data = np.array(imageFeatures_data)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y = image_labels_data

# Split our model into a test and training dataset to train our LR classifier
X_train, X_test, y_train, y_test = train_test_split(image_features_data, y, test_size=0.2, random_state = 7)

glm = LogisticRegression(C=0.1)
glm.fit(X_train,y_train)

LogisticRegression(C=0.1)

In [ ]:
# Get Accruacy on the 20% we split from our training dataset
accuracy = glm.score(X_test, y_test)
print(f'Accuracy on validation set using Logistic Regression: {accuracy*100}%')

Accuracy on validation set using Logistic Regression: 100.0%


## **5. Test some inferences**

In [ ]:
image_names_test = os.listdir("./test1")
image_paths_test = ["./test1/"+ x for x in image_names_test]

In [ ]:
import random

test_sample = random.sample(image_paths_test, 12)

def test_img():
    result_lst = []
    for path in test_sample:
        image = load_img(path, target_size=(224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)
        features = model.predict(image)
        features = np.reshape(features,(-1,7*7*512))
        result = glm.predict(features)
        result = 'dog' if float(result) >0.5 else 'cat'
        result_lst.append(result)
    return result_lst

In [ ]:
# get test predictions from all models
pred_results = test_img()
pred_results

1/1 [==============================] - 0s 46ms/step


['dog',
 'dog',
 'cat',
 'cat',
 'cat',
 'cat',
 'cat',
 'cat',
 'dog',
 'cat',
 'cat',
 'cat']

In [ ]:
plt.figure(figsize=(15, 15))

for i in range(0, 12):
    plt.subplot(4, 3, i+1)
    result = pred_results[i]
    img = test_sample[i]
    image = load_img(img, target_size=(256,256))
    plt.text(72, 248, f'Feature Extractor CNN: {result}', color='lightgreen',fontsize= 12, bbox=dict(facecolor='black', alpha=0.9))
    plt.imshow(image)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

## **How do we compare to Kaggle's top 10?**
https://www.kaggle.com/c/dogs-vs-cats/leaderboard
